In [6]:
# Cargar el paquete LPE
library(LPE)


In [7]:
# Leer sin asignar row.names para evitar error
data_raw <- read.csv("data1-raw-test.csv", stringsAsFactors = FALSE)

# Revisar la columna de genes (suponiendo que es la primera columna)
head(data_raw[,1])

# Quitar filas con NA en la columna de genes
data_raw <- data_raw[!is.na(data_raw[,1]), ]

# Ordenar y eliminar duplicados conservando la fila con mayor expresión promedio
data_raw$mean_expr <- rowMeans(data_raw[,-1])
data_raw <- data_raw[order(data_raw$mean_expr, decreasing = TRUE), ]
data_raw <- data_raw[!duplicated(data_raw[,1]), ]

# Ahora asignar los nombres de genes como rownames
rownames(data_raw) <- data_raw[,1]

# Eliminar la columna de genes que ya está como rownames
data_raw <- data_raw[,-1]

[1] NA          "LINC02615" "LOC145474" "LINP1"     "KIAA0040"  "LINC02541"

In [8]:
# Quitar genes con NA en rownames si existen
data_raw <- data_raw[!is.na(rownames(data_raw)), ]

# Si hay duplicados en rownames, conservar solo uno (el de mayor expresión media, por ejemplo)
data_raw$mean_expr <- rowMeans(data_raw)
data_raw <- data_raw[order(data_raw$mean_expr, decreasing = TRUE), ]
data_raw <- data_raw[!duplicated(rownames(data_raw)), ]
data_raw$mean_expr <- NULL

In [9]:
# Seleccionar solo las columnas WT y TLR4 para el análisis
expr_WT <- data_raw[, c("WT_1.gProcessedSignal", "WT_2.gProcessedSignal")]
expr_TLR4 <- data_raw[, c("RNA5_TLR4_1.gProcessedSignal", "RNA5_TLR4_2.gProcessedSignal")]

In [10]:
# Calcular baseline variance con baseOlig.error
bv_WT <- baseOlig.error(expr_WT)
bv_TLR4 <- baseOlig.error(expr_TLR4)


In [11]:
lpe_result <- lpe(x = expr_WT,
                  y = expr_TLR4,
                  basevar.x = bv_WT,
                  basevar.y = bv_TLR4,
                  probe.set.name = rownames(data_raw))

In [12]:
# Este objeto ya es un data.frame con p-values, fold changes, etc.
head(lpe_result)

# Guardar resultados
write.csv(lpe_result, "LPE_results_WT_vs_TLR4.csv")

,x.WT_1.gProcessedSignal,x.WT_2.gProcessedSignal,median.1,std.dev.1,p.outlier.x,flag.outlier.x,y.RNA5_TLR4_1.gProcessedSignal,y.RNA5_TLR4_2.gProcessedSignal,median.2,std.dev.2,p.outlier.y,flag.outlier.y,median.diff,pooled.std.dev,z.stats
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>
RPS2,312740.3,281750.5,297245.4,12256.87,0.07380487,.,272843.4,296985.9,284914.7,8987.626,0.07945264,.,12330.75,10747.29,1.1473360
RNA28SN1,234254.5,243495.0,238874.8,12256.87,0.61563798,.,243192.3,257579.5,250385.9,8987.626,0.25222861,.,-11511.15,10747.29,-1.0710749
lnc-WWC2-1,240595.2,236774.1,238684.7,12256.87,0.83564162,.,233754.2,231623.5,232688.9,8987.626,0.86166166,.,5995.80,10747.29,0.5578896
GPR155,225910.3,212002.4,218956.3,12256.87,0.47694701,.,234068.5,233376.8,233722.6,8987.626,0.95494236,.,-14766.30,10747.29,-1.3739560
RPS20,205117.3,229211.5,217164.4,12256.87,0.22016529,.,231484.0,224028.4,227756.2,8987.626,0.53990695,.,-10591.80,10747.29,-0.9855324
ZNF865,211529.2,208909.5,210219.4,13047.50,0.89585597,.,231812.5,237551.9,234682.2,8987.626,0.63959135,.,-24462.85,11203.01,-2.1835970


In [13]:
lpe_result$p.value <- 2 * pnorm(-abs(lpe_result$z.stats))

In [14]:
sig_genes <- subset(lpe_result, p.value < 0.05)

In [15]:
sig_genes_fc <- subset(sig_genes, abs(median.diff) > 1.5)

In [16]:
head(sig_genes_fc)

,x.WT_1.gProcessedSignal,x.WT_2.gProcessedSignal,median.1,std.dev.1,p.outlier.x,flag.outlier.x,y.RNA5_TLR4_1.gProcessedSignal,y.RNA5_TLR4_2.gProcessedSignal,median.2,std.dev.2,p.outlier.y,flag.outlier.y,median.diff,pooled.std.dev,z.stats,p.value
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
ZNF865,211529.2,208909.5,210219.4,13047.50,0.8958560,.,231812.5,237551.9,234682.2,8987.626,0.6395914,.,-24462.85,11203.01,-2.183597,2.899188e-02
TONSL,197017.1,222607.6,209812.4,13134.69,0.2014762,.,242029.8,235365.4,238697.6,8987.626,0.5886208,.,-28885.25,11253.83,-2.566703,1.026706e-02
CC2D1A,197852.0,229764.2,213808.1,12256.87,0.1075261,.,233482.1,248105.8,240794.0,8987.626,0.2367627,.,-26985.85,10747.29,-2.510945,1.204084e-02
PTK6,179682.2,195957.9,187820.0,15700.20,0.4395143,.,207178.8,222545.9,214862.3,8987.626,0.2023549,.,-27042.30,12792.06,-2.113991,3.451606e-02
ACTB,216729.7,209739.2,213234.5,12331.92,0.7248245,.,167873.3,170117.1,168995.2,8035.283,0.8537671,.,44239.25,10407.74,4.250612,2.131868e-05
RPS8,172663.3,189881.9,181272.6,15944.24,0.4193413,.,217113.7,207095.4,212104.5,8902.220,0.4053768,.,-30831.95,12912.56,-2.387750,1.695189e-02


In [17]:
write.csv(sig_genes_fc, "DEG_LPE_WT_vs_TLR4.csv")

In [18]:
# Suponiendo lpe_result tiene rownames con nombres de genes
genes <- rownames(lpe_result)

# Filtrar genes que NO comienzan con "lnc" ni "XLOC" (ejemplo de nombre raro)
keep <- !grepl("^(lnc|XLOC|.*_AS1|.*-AS1)$", genes, ignore.case = TRUE)

# Filtrar el dataframe
lpe_filtered <- lpe_result[keep, ]

# Revisar resultado
head(lpe_filtered)

,x.WT_1.gProcessedSignal,x.WT_2.gProcessedSignal,median.1,std.dev.1,p.outlier.x,flag.outlier.x,y.RNA5_TLR4_1.gProcessedSignal,y.RNA5_TLR4_2.gProcessedSignal,median.2,std.dev.2,p.outlier.y,flag.outlier.y,median.diff,pooled.std.dev,z.stats,p.value
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
RPS2,312740.3,281750.5,297245.4,12256.87,0.07380487,.,272843.4,296985.9,284914.7,8987.626,0.07945264,.,12330.75,10747.29,1.1473360,0.25124277
RNA28SN1,234254.5,243495.0,238874.8,12256.87,0.61563798,.,243192.3,257579.5,250385.9,8987.626,0.25222861,.,-11511.15,10747.29,-1.0710749,0.28413575
lnc-WWC2-1,240595.2,236774.1,238684.7,12256.87,0.83564162,.,233754.2,231623.5,232688.9,8987.626,0.86166166,.,5995.80,10747.29,0.5578896,0.57691977
GPR155,225910.3,212002.4,218956.3,12256.87,0.47694701,.,234068.5,233376.8,233722.6,8987.626,0.95494236,.,-14766.30,10747.29,-1.3739560,0.16945534
RPS20,205117.3,229211.5,217164.4,12256.87,0.22016529,.,231484.0,224028.4,227756.2,8987.626,0.53990695,.,-10591.80,10747.29,-0.9855324,0.32436262
ZNF865,211529.2,208909.5,210219.4,13047.50,0.89585597,.,231812.5,237551.9,234682.2,8987.626,0.63959135,.,-24462.85,11203.01,-2.1835970,0.02899188


In [21]:
genes <- rownames(lpe_filtered)

# Patrón para eliminar LOC
pattern_exclude <- "^(LOC)"

keep_final <- !grepl(pattern_exclude, genes, ignore.case = TRUE)

lpe_final <- lpe_filtered[keep_final, ]

# Verificar
head(rownames(lpe_final))

[1] "RPS2"       "RNA28SN1"   "lnc-WWC2-1" "GPR155"     "RPS20"     
[6] "ZNF865"

In [22]:
# Suponiendo que 'lpe_final' es tu dataframe final filtrado de genes codificantes
num_genes <- nrow(lpe_final)
cat("Número total de genes codificantes en el análisis:", num_genes, "\n")

Número total de genes codificantes en el análisis: 29546 
